# Dane wejściowe oraz algorytm KMP

In [455]:
data = [
    "bbb$",
    "aabbabd$",
    "ababcd$",
    "abaababaabaabaabab$",
    "Jest piękna pogoda na spacer po parku. Słońce świeci, ptaki śpiewają, a ludzie się uśmiechają. Wspan$",
]
# with open("lab6 - suffix tree/1997_714_head.txt", "r") as file:
with open("1997_714_head.txt", "r") as file:
    data.append(file.read() + "$")

def prefixes_sufixes(text: str):
    ans = []
    for i in range(len(text)):
        ans.append(text[i:])
        ans.append(text[:-i])
    return ans

def prefixFunction(P):
    res = [0]
    k = 0
    for j in range(1, len(P)):
        while k > 0 and P[k] != P[j]:
            k = res[k-1]
        if P[k] == P[j]:
            k += 1
        res.append(k)
    return res


def _findPatternKMP(T, P, pi):
    S = []
    m = len(P)
    q = 0
    for i in range(len(T)):
        while q > 0 and T[i] != P[q]:
            q = pi[q-1]
        if T[i] == P[q]:
            q += 1 
        if q == m:
            q = pi[q-1]
            S.append(i - m + 1)
    return S

def findPattern(T, P):
    pi = prefixFunction(P)
    return _findPatternKMP(T, P, pi)


# Trie zawierające wszystkie sufixy

In [458]:
class SufixesTrie:
    def __add_string(self, string: str) -> None:
        walker = self.root
        for i, letter in enumerate(string):
            if not walker.is_conected(letter):
                walker.add_child(letter)
                walker = walker.get_child(letter)
                for _letter in string[i+1:]:
                    walker.add_child(_letter)
                    walker = walker.get_child(_letter)
                break
            walker = walker.get_child(letter)

    def __init__(self, data: str) -> None:
        self.root = SufixesTrieNode()
        tmp_str = data
        while tmp_str[1:]:
            self.__add_string(tmp_str)
            tmp_str = tmp_str[1:]
        self.__add_string(tmp_str)

    def __contains__(self, item):
        walker = self.root
        for letter in item:
            if walker.is_conected(letter):
                walker = walker.get_child(letter)
            else:
                return False
        return True

class SufixesTrieNode:
    def __init__(self) -> None:
        self.children = dict()
    
    def get_child(self, branch_str: str):
        if self.is_conected(branch_str):
            return self.children[branch_str]
        else:
            raise ValueError(f"NOT SUCH BRANCH: {branch_str}")
    
    def add_child(self, branch_str: str):
        if self.is_conected(branch_str):
            raise ValueError("SUCH CHILD ALREADY EXIST")
        else:
            self.children[branch_str] = SufixesTrieNode()
    
    def is_conected(self, branch_str: str):
        return branch_str in self.children

In [460]:
for text in data:
    print(text)
    patterns = prefixes_sufixes(text)
    trie = SufixesTrie(text)

    for pattern in patterns:
        pattern in trie

bbb$
aabbabd$
ababcd$
abaababaabaabaabab$
Jest piękna pogoda na spacer po parku. Słońce świeci, ptaki śpiewają, a ludzie się uśmiechają. Wspan$




Dz.U. z 1998 r. Nr 144, poz. 930
                                       
                                       
                                       
                                       
                                    USTAWA
                          z dnia 20 listopada 1998 r.
                                       
         o zryczaĹ‚towanym podatku dochodowym od niektĂłrych przychodĂłw
                        osiÄ…ganych przez osoby fizyczne
                                       
                                  RozdziaĹ‚ 1
                                Przepisy ogĂłlne
                                       
                                    Art. 1.
Ustawa reguluje opodatkowanie zryczaĹ‚towanym podatkiem dochodowym niektĂłrych
przychodĂłw (dochodĂłw) osiÄ…ganych przez osoby fizyczne prowadzÄ…ce pozarolniczÄ…
dziaĹ‚alnoĹ

# Trie budowane on line (z suffix link'ami)

In [411]:
class OnLineTrie:
    def __build_tree(self, string: str):
        depest = self.root

        for letter in string:
            node = depest
            depest = None
            prev = None

            while not(node.is_conected(letter)):
                node.add_child(letter)
                child = node.get_child(letter)

                if depest is None:
                    depest = child
                else:
                    prev.link(child)
                
                if(node == self.root):
                    child.link(self.root)
                
                prev = child
                if node.is_linked():
                    node = node.get_link()
                else:
                    node = None
                    break
            
            if node:
                prev.link(node.add_child(letter))

    def __init__(self, string: str) -> None:
        self.root = OnLineTrieNode()
        self.__build_tree(string)

    def __contains__(self, item):
        walker = self.root
        for letter in item:
            if walker.is_conected(letter):
                walker = walker.get_child(letter)
            else:
                return False
        return True

class OnLineTrieNode:
    def __init__(self) -> None:
        self.children = dict()
        self.up_link = None
    
    def get_child(self, branch_str: str):
        if self.is_conected(branch_str):
            return self.children[branch_str]
        else:
            raise ValueError(f"NOT SUCH BRANCH: {branch_str}")
    
    def add_child(self, branch_str: str):
        if self.is_conected(branch_str):
            raise ValueError("SUCH CHILD ALREADY EXIST")
        else:
            self.children[branch_str] = OnLineTrieNode()
    
    def is_conected(self, branch_str):
        return branch_str in self.children
    
    def link(self, node):
        if isinstance(node, OnLineTrieNode):
            self.link = node
        else:
            raise ValueError(f"WRONG ISTANCE OF UP_LINK: {type(node)}")
    
    def is_linked(self):
        return self.up_link is not None
    
    def get_link(self):
        if self.is_linked():
            return self.link
        else:
            raise RuntimeError("THIS NODE DONT HAVE LINK")

In [412]:
print(data[2])
trie = OnLineTrie(data[2])

print("ab" in trie)


ababcd$
True


# suffix tree

In [413]:
class ActivePoint:
    def __init__(self, node) -> None:
        # węzeł na którym aktualnie pracujemy
        self.node = node
        # pierwsza litera aktywnej krawędiz, None -> będziemy dodawać nową krawędź
        self.edge = None
        # odległość od aktytwnej krawędzi gdzie potencjalnie nastąpi rozgałęzienie
        self.length = 0
        # liczba pominiętych sufixów
        self.reminder = 0
    
    def __change_node(self, node, text):
        # print(f"\t\tnew_node: {node},{node.string_edge_above(text)}")
        if isinstance(node, SuffixTreeNode):
            self.node = node
        else:
            raise ValueError(f"WRONG NODE TYPE: {type(node)}")
        
    def __change_edge(self, edge):
        # print(f"\t\tedge changed {edge}")
        if edge is None:
            self.edge = None
        elif isinstance(edge, str):
            self.edge = edge
        else:
            raise ValueError(f"WRONG EDGE TYPE {type(edge)}")
    
    def __change_length(self, length, text):
        # print(f"\t\tchanged len {length}\t current_line ")
        if length < 0:
            raise ValueError(f"LENGTH BELOW 0: {length}")
        else:
            self.length = length

    def __line_indexes(self):
        if self.edge is None:
            return (0, 0)
        if self.node.is_contain(self.edge):
            child = self.node.get_child(self.edge)
        else:
            raise RuntimeError("EDGE IS NOT AN EDGE IN TREE")
        return (child.parent_index, child.parent_index + self.length)

    def __line_str(self, text):
        if self.edge is None:
            return ""
        start, end = self.__line_indexes()
        return f"{self.node} -> {self.edge}:{text[start: end]}"

    def on_path(self, letter, text):
        if letter not in self.node.children:
            return False
        else:
            if self.edge is None:
                self.__change_edge(letter)
            return letter == text[self.node.get_child(self.edge).parent_index + self.length]
    
    def extend(self, letter, text, i):
        if self.edge is None:
            self.__change_edge(letter)
        self.__change_length(self.length + 1, text)
        self.reminder += 1
        self.__adapt(text, i)
        # print(f"\textend\tcurrent line: {self.__line_str(text)}")

    def __adapt(self, text, i):
        if self.edge is None:
            return
        child = self.node.get_child(self.edge)
        child_length_from_parent = child.length_edge_above(text)
        # print(f"\t\t adapt\t{child_length_from_parent}\t{self.__line_str(text)}")
        while child_length_from_parent < self.length:
            # print(f"\t\tjump")
            self.__change_edge(text[i - self.length + child_length_from_parent])
            self.node = child
            self.__change_length(self.length - child_length_from_parent, text)

            if self.node.is_contain(self.edge):
                child = self.node.get_child(self.edge)
                child_length_from_parent = child.length_edge_above(text)
            else:
                child.add_child(self.edge,-1,i-self.length)
                self.__change_edge(None)
                break

        if child_length_from_parent == self.length:
            # print(f"\t\tjump0")
            self.__change_node(child, text)
            self.__change_edge(None)
            self.__change_length(0,text)

    def add_child(self, letter, text, i):
        if self.node.is_contain(self.edge):
            child = self.node.divine_edge(letter, self.length, self.edge, text, i)
        else:
            self.node.add_child(letter, -1, i)
            child = self.node
            # print(f"\tchild:{letter}, {self.node.get_child(letter)} added to {self.node}")
        return child
    
    def step(self, text, i):
        self.reminder -= 1
        if self.node.suffix_link == self.node:
            if self.length > 0:
                self.__change_length(self.length - 1, text)
                new_edge = text[i-self.length]
                if self.node.is_contain(new_edge):
                    self.__change_edge(new_edge)
                else:
                    self.__change_edge(None)
                # print(f"\tstep\tcurrent line: {self.__line_str(text)}")
            else:
                self.__change_edge(None)
            ans = self.node
            self.__adapt(text, i)

        else:
            self.__change_node(self.node.suffix_link, text)
            ans = self.node
            self.__adapt(text, i)
        return ans

class SuffixTree:
    def __init__(self, text: str) -> None:
        self.text = text
        self.root = SuffixTreeNode(0,None)
        self.root.link(self.root)
        self.__build_tree()

    def __build_tree(self):
        n = len(self.text)
        active_point = ActivePoint(self.root)
        for i in range(n):
            # if n - i > 10:
            #     print(f"{self.text[i:i+6]}...")
            # else:
            #     print(self.text[i:])
            letter = self.text[i]

            if active_point.on_path(letter, self.text):
                active_point.extend(letter, self.text, i)
            else:
                prev_child = active_point.add_child(letter, self.text, i)
                while active_point.reminder > 0:
                    node = active_point.step(self.text, i)
                    if active_point.on_path(letter, self.text):
                        prev_child.link(active_point.node)
                        active_point.extend(letter, self.text, i)
                        break
                    child = active_point.add_child(letter, self.text, i)
                    prev_child.link(child)

                    prev_child = child
    
    def __contains__(self, item):
        n = len(item)
        i = 0
        walker = self.root
        while i < n:
            edge_letter = item[i]
            if walker.is_contain(edge_letter):
                walker = walker.get_child(edge_letter)
                start, end = walker.text_indexes(self.text)
                for j in range(start, end):
                    if i >= n:
                        return True
                    elif item[i] == self.text[j]:
                        i += 1
                    else:
                        return False
            else:
                return False
        return True

class SuffixTreeNode:
    def __init__(self, index, parent_index) -> None:
        self.children = dict()
        # gałęzie odpowiadają za łańcuchy znaków - w węzłach są przechowywane informacje 
        # jakiemu łąńcuchowi znaków odpoiada krawędź od siebie samego do swojego ojca 
        # indeks znaku za którego koniec odpowiada węzeł,
        #   index == -1 -> mamy na myśli ostatni index
        self.index = index
        #   parent_index == None -> mamy do czynienia z root'em
        #   index == 0 -> mamy do czynienia z root'em
        self.parent_index = parent_index

        if parent_index is not None and index != 0 and index != -1:
            if index - parent_index <= 0:
                raise ValueError(f"BAD INDEXES {self}")

        self.suffix_link = None
    
    def add_child(self, first_letter: str, index, parent_index):
        # print(f"\t\tadded child {first_letter}, [{parent_index}, {index})")
        if self.is_contain(first_letter):
            raise ValueError(F"SUCH CHILD ALREADY IN TREE: {first_letter}")
        else:
            self.children[first_letter] = SuffixTreeNode(index, parent_index)

    def get_child(self, first_letter: str):
        if self.is_contain(first_letter):
            return self.children[first_letter]
        else:
            raise ValueError(f"NOT SUCH CHILD: {first_letter}")
    
    def divine_edge(self, letter, length, edge, text, i):
        old_child = self.get_child(edge)
        child_index = old_child.parent_index + length
        child = SuffixTreeNode(parent_index=old_child.parent_index, index=child_index)

        self.children[edge] = child
        old_child.parent_index = child_index
        child.add_child(letter, -1, i)
        child.children[text[child_index]] = old_child

        # print(f"\tdivined edge\tfrom: {self} created: {edge}, {child}, with children: {child.children}")
        return child

    
    def link(self, other):
        # print(f"\t\t{self} linked to {other}")
        if isinstance(other, SuffixTreeNode):
            self.suffix_link = other
        else:
            raise ValueError(f"WRONG INSTANCE OF LINKING ITEM {type(other)}")

    def is_contain(self, first_letter: str):
        return first_letter in self.children
    
    # wypisuje tekst za który odpowiada gałąź od ojca do tego węzłą
    def string_edge_above(self, text):
        if self.index == -1:
            return text[self.parent_index:]
        elif self.parent_index == None:
            return ""
        else:
            return text[self.parent_index:self.index]
    
    def text_indexes(self, text):
        if self.parent_index == None:
            return None
        elif self.index == -1:
            return self.parent_index, len(text)
        else:
            return self.parent_index, self.index
    
    # zwraca długość na gałęzi między sobą a swoim ojcem
    def length_edge_above(self, text):
        if self.index == -1:
            ans = len(text) - self.parent_index
        elif self.parent_index == None:
            return None
        else:
            ans = self.index - self.parent_index 
        if ans < 0:
            raise ValueError("EDGE LENGTH ABOVE ERROR")
        return ans
    
    def __str__(self) -> str:
        return f"[{self.parent_index}, {self.index})"
        
    def __repr__(self) -> str:
        return self.__str__()



In [469]:
def test_prefix_tree(text):
    try:
        first_lecture_tree = SuffixTree(text)
    except ValueError as e:
        print(f"Value Error Creating Tree: {e}")
        return
    except RuntimeError as e:
        print(f"Runtime Error Cerating Tree: {e}")
        return
    
    Wrongs = []
    for i in range(len(text)):
        if not text[i:] in first_lecture_tree:
            Wrongs.append(text[i:])
        if not text[:-i] in first_lecture_tree:
            Wrongs.append(text[:-1])
    if len(Wrongs) > 0:
        print(f"Invalid Tree:{Wrongs} this patterns should be in tree")
    else:
        print("PASS")

# test_prefix_tree(data[5])

for text in data:
    test_prefix_tree(text)
# Problemem jest najprawdopodoniej tworzenie suffix linków (reguła 2)

PASS
PASS
PASS
PASS
Invalid Tree:['ają. Wspan$', 'span$'] this patterns should be in tree
Value Error Creating Tree: WRONG NODE TYPE: <class 'NoneType'>


# Porównanie Struktór

In [468]:
from prettytable import PrettyTable
from horology import Timing, tformatter

def get_time_str(interval, unit):
    time = tformatter.rescale_time(interval, unit)
    return f"{time[0]:3g} {time[1]}"

def my_add_row(table:PrettyTable, algorithm_name:str, prep:Timing, find:Timing):
    table.add_row(
        [
            algorithm_name, 
            get_time_str(prep.interval, prep.unit), 
            get_time_str(find.interval, find.unit), 
            get_time_str(prep.interval + find.interval, prep.unit)
        ])

Tables = []
filds_names = ["algorytm","preprocesing", "czas wyszukiwania", "suma"]

Trees = [("Trie suffix'ów", SufixesTrie), ("suffix link Trie", OnLineTrie), ("drzewo suffix'ów", SuffixTree)]

for text in data:
    table = PrettyTable()
    Tables.append(table)
    if len(text) < 20:
        table.title = text
    else:
        table.title = f"tekst o długości: {len(text)}"
    table.field_names = filds_names

    patterns = prefixes_sufixes(text)
    
    # KMP
    with Timing(print_fn=None) as preprocess:
        p_funcs = [prefixFunction(pattern) for pattern in patterns]
    
    with Timing(print_fn=None) as finding:
        for i, pattern in enumerate(patterns):
            _findPatternKMP(text, patterns, p_funcs[i])
    
    my_add_row(table, "KMP", preprocess, finding)
    
    # trees
    for algo_name, structure in Trees:
        try:
            with Timing(print_fn=None) as preprocess:
                tree = structure(text)
            with Timing(print_fn=None) as findin:
                for pattern in patterns:
                    pattern in tree
            my_add_row(table, algo_name, preprocess, finding)
        except:
            continue

for table in Tables:
    print(table)
    print()
    print()

+------------------------------------------------------------------+
|                               bbb$                               |
+------------------+--------------+-------------------+------------+
|     algorytm     | preprocesing | czas wyszukiwania |    suma    |
+------------------+--------------+-------------------+------------+
|       KMP        |  36.5496 ms  |      44.9 us      | 36.5945 ms |
|  Trie suffix'ów  |   43.2 us    |      44.9 us      |  88.1 us   |
| suffix link Trie |   25.9 us    |      44.9 us      |  70.8 us   |
| drzewo suffix'ów |   65.4 us    |      44.9 us      |  110.3 us  |
+------------------+--------------+-------------------+------------+


+----------------------------------------------------------------+
|                            aabbabd$                            |
+------------------+--------------+-------------------+----------+
|     algorytm     | preprocesing | czas wyszukiwania |   suma   |
+------------------+--------------+-----

# Porównanie czasów:
Czasy wyszukiwania są bardzo podobne, nawet pomiędzy KMP a strukturami drzewistymico jest spowodowane 
najprawdopodobniej tym że jest to suma czasó wyszukiwania wszystkich możliwych zawierających się wzorców
co daję oszacowaną długość wzorca O(2n) gdzie n to długość tekstu  
Dla krótszych wzorców czasy preprocesingu są podobne, najlepiej sobie radzi algorytm KMP  
Dla dłuższych wzorców czas preprocesingu są największa dla najprostszej implementacji drzewa trie  
<!--  -->

# Przykładowe wyniki
## bbb$

| algorytm            | preprocesing | czas wyszukiwania | suma      |
|---------------------|--------------|------------------|-----------|
| KMP                 | 32.3792 ms   | 46.2 us          | 32.4254 ms|
| Trie suffix'ów      | 44.4 us      | 46.2 us          | 90.6 us  |
| suffix link Trie    | 27.2 us      | 46.2 us          | 73.4 us  |
| drzewo suffix'ów   | 186.4 us     | 46.2 us          | 232.6 us |

## aabbabd$

| algorytm            | preprocesing | czas wyszukiwania | suma     |
|---------------------|--------------|------------------|----------|
| KMP                 | 37.2 us      | 42.3 us          | 79.5 us  |
| Trie suffix'ów      | 65.4 us      | 42.3 us          | 107.7 us |
| suffix link Trie    | 149.6 us     | 42.3 us          | 191.9 us |
| drzewo suffix'ów   | 76.4 us      | 42.3 us          | 118.7 us |

## ababcd$

| algorytm            | preprocesing | czas wyszukiwania | suma     |
|---------------------|--------------|------------------|----------|
| KMP                 | 32.8 us      | 39 us            | 71.8 us  |
| Trie suffix'ów      | 60.4 us      | 39 us            | 99.4 us  |
| suffix link Trie    | 26.6 us      | 39 us            | 65.6 us  |
| drzewo suffix'ów   | 60 us        | 39 us            | 99 us    |

## abaababaabaabaabab$

| algorytm            | preprocesing | czas wyszukiwania | suma     |
|---------------------|--------------|------------------|----------|
| KMP                 | 249.6 us     | 286 us           | 535.6 us |
| Trie suffix'ów      | 380.1 us     | 286 us           | 666.1 us |
| suffix link Trie    | 151.7 us     | 286 us           | 437.7 us |
| drzewo suffix'ów   | 301.9 us     | 286 us           | 587.9 us |

## tekst o długości: 101

| algorytm            | preprocesing | czas wyszukiwania | suma      |
|---------------------|--------------|------------------|-----------|
| KMP                 | 4.8055 ms    | 44.9258 ms       | 49.7313 ms|
| Trie suffix'ów      | 11.0657 ms   | 44.9258 ms       | 55.9915 ms|
| suffix link Trie    | 1.3757 ms    | 44.9258 ms       | 46.3015 ms|
| drzewo suffix'ów   | 1.4237 ms    | 44.9258 ms       | 46.3495 ms|

## tekst o długości: 2539

| algorytm            | preprocesing | czas wyszukiwania | suma     |
|---------------------|--------------|------------------|----------|
| KMP                 | 1.65521 s    | 1.91699 s        | 3.5722 s |
| Trie suffix'ów      | 5.54256 s    | 1.91699 s        | 7.45955 s|
| suffix link Trie    | 325.392 ms   | 1.91699 s        | 2.24238 s|


# Uwagi
drzewo suffix'ów dla przedostatniego tekstu zostało źle stworzone - nie ma w nim wszystkich gałęzi  
drzewo suffix'ów dla ostatniego tekstu wogule się nie wygenerowało  
  
najprowdopodobniej błąd implementacyjny dotyczy tworzenia suffix link'ów